## Easter Weather in Switzerland

In [225]:
import numpy as np
import pandas as pd
from plotnine import *

pd.set_option('display.max.columns', 500)

In [143]:
links = pd.read_csv("liste-download-nbcn-d.csv", encoding='latin')

In [144]:
links.head(2)

,Station,station/location,WIGOS-ID,Data since,Station height m. a. sea level,CoordinatesE,CoordinatesN,Latitude,Longitude,Climate region,Canton,URL Previous years (verified data),URL Current year
0,Altdorf,ALT,0-20000-0-06672,01.01.1864,438,2690181,1193564,46.887069,8.621894,Central Alpine north slope,UR,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...
1,Andermatt,ANT,0-20000-0-06695,01.01.1864,1438,2687445,1165044,46.630914,8.580553,Central Alpine north slope,UR,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...


Which stations do we have?

In [145]:
links.value_counts('Canton').index

Index(['GR', 'VS', 'BE', 'NE', 'SG', 'TI', 'UR', 'VD', 'AI', 'BL', 'GE', 'GL',
       'LU', 'OW', 'ZH'],
      dtype='object', name='Canton')

In [146]:
links.value_counts('Station').index

Index(['Altdorf', 'La Chaux-de-Fonds', 'Säntis', 'St. Gallen', 'Sion',
       'Segl-Maria', 'Samedan', 'S. Bernardino', 'Payerne', 'Neuchâtel',
       'Meiringen', 'Luzern', 'Lugano', 'Locarno / Monti', 'Jungfraujoch',
       'Andermatt', 'Grächen', 'Grimsel Hospiz', 'Genève / Cointrin',
       'Engelberg', 'Elm', 'Davos', 'Col du Grand St-Bernard', 'Château-d'Oex',
       'Chaumont', 'Bern / Zollikofen', 'Basel / Binningen', 'Bad Ragaz',
       'Zürich / Fluntern'],
      dtype='object', name='Station')

In [147]:
links = (links
 .query("Station in ['La Chaux-de-Fonds', 'Säntis', 'St. Gallen', 'Zürich / Fluntern']")
 .reset_index(drop=True)
 )

links

,Station,station/location,WIGOS-ID,Data since,Station height m. a. sea level,CoordinatesE,CoordinatesN,Latitude,Longitude,Climate region,Canton,URL Previous years (verified data),URL Current year
0,La Chaux-de-Fonds,CDF,0-20000-0-06612,01.01.1900,1017,2550919,1214862,47.082947,6.792314,Western Jura,NE,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...
1,Säntis,SAE,0-20000-0-06680,01.01.1864,2501,2744188,1234920,47.249447,9.343469,Eastern Alpine north slope,AI,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...
2,Zürich / Fluntern,SMA,0-20000-0-06660,01.01.1864,556,2685118,1248066,47.377925,8.565742,North-eastern plateau,ZH,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...
3,St. Gallen,STG,0-20000-0-06681,01.01.1864,776,2747866,1254588,47.425475,9.398528,North-eastern plateau,SG,https://data.geo.admin.ch/ch.meteoschweiz.klim...,https://data.geo.admin.ch/ch.meteoschweiz.klim...


I now want to download the csv files from the links. I can store the resulting dataframes as dictionaries in a new column:

In [148]:
def download_data(url):
    df = pd.read_csv(url, delimiter=';', parse_dates=['date'])
    return df.to_dict()

In [153]:
downloaded_data = (links
                   .assign(historical_data=lambda x: x['URL Previous years (verified data)'].apply(download_data),
                           present_data=lambda x: x['URL Current year'].apply(download_data))
                   .filter(['Station', 'station/location', 'historical_data', 'present_data'])
                   )

downloaded_data

,Station,station/location,historical_data,present_data
0,La Chaux-de-Fonds,CDF,"{'station/location': {0: 'CDF', 1: 'CDF', 2: '...","{'station/location': {0: 'CDF', 1: 'CDF', 2: '..."
1,Säntis,SAE,"{'station/location': {0: 'SAE', 1: 'SAE', 2: '...","{'station/location': {0: 'SAE', 1: 'SAE', 2: '..."
2,Zürich / Fluntern,SMA,"{'station/location': {0: 'SMA', 1: 'SMA', 2: '...","{'station/location': {0: 'SMA', 1: 'SMA', 2: '..."
3,St. Gallen,STG,"{'station/location': {0: 'STG', 1: 'STG', 2: '...","{'station/location': {0: 'STG', 1: 'STG', 2: '..."


It seems I need a for loop for the unnesting here...

![](https://media.tenor.com/MRk0G7hEXzAAAAAM/disgusted-disappointed.gif)

In [155]:
downloaded_data = (downloaded_data
                 .melt(id_vars=['Station', 'station/location'])
                 )

downloaded_data

,Station,station/location,variable,value
0,La Chaux-de-Fonds,CDF,historical_data,"{'station/location': {0: 'CDF', 1: 'CDF', 2: '..."
1,Säntis,SAE,historical_data,"{'station/location': {0: 'SAE', 1: 'SAE', 2: '..."
2,Zürich / Fluntern,SMA,historical_data,"{'station/location': {0: 'SMA', 1: 'SMA', 2: '..."
3,St. Gallen,STG,historical_data,"{'station/location': {0: 'STG', 1: 'STG', 2: '..."
4,La Chaux-de-Fonds,CDF,present_data,"{'station/location': {0: 'CDF', 1: 'CDF', 2: '..."
5,Säntis,SAE,present_data,"{'station/location': {0: 'SAE', 1: 'SAE', 2: '..."
6,Zürich / Fluntern,SMA,present_data,"{'station/location': {0: 'SMA', 1: 'SMA', 2: '..."
7,St. Gallen,STG,present_data,"{'station/location': {0: 'STG', 1: 'STG', 2: '..."


In [157]:
dfs = []

for row in range(downloaded_data.shape[0]):
    df = pd.DataFrame(downloaded_data.iloc[row, 3])
    dfs.append(df)

And apparently these are the units, so the next step is concatenating the list of dataframes and then renaming a little:

Parameter            Einheit          Beschreibung
gre000d0             W/m²             Globalstrahlung; Tagesmittel
hto000d0             cm               Gesamtschneehöhe; Morgenmessung von 6 UTC
nto000d0             %                Gesamtbewölkung; Tagesmittel
prestad0             hPa              Luftdruck auf Stationshöhe (QFE); Tagesmittel
rre150d0             mm               Niederschlag; Tagessumme 6 UTC - 6 UTC Folgetag
sre000d0             min              Sonnenscheindauer; Tagessumme
tre200d0             °C               Lufttemperatur 2 m über Boden; Tagesmittel
tre200dn             °C               Lufttemperatur 2 m über Boden; Tagesminimum
tre200dx             °C               Lufttemperatur 2 m über Boden; Tagesmaximum
ure200d0             %                Relative Luftfeuchtigkeit 2 m über Boden; Tagesmittel

In [165]:
data = (pd.concat(dfs)
 .merge((links.filter(['Station', 'station/location'])))
 .drop(['gre000d0', 'hto000d0', 'prestad0', 
        'tre200dn', 'tre200dx', 'ure200d0'], axis=1)
 .rename(columns={'nto000d0': 'Bewoelkung',
                  'rre150d0': 'Niederschlag',
                  'sre000d0': 'Sonnenscheindauer',
                  'tre200d0': 'Temperatur'})
 .assign(year = lambda x: x['date'].dt.year,
         month = lambda x: x['date'].dt.month)
 )

data

,station/location,date,Bewoelkung,Niederschlag,Sonnenscheindauer,Temperatur,Station,year,month
0,CDF,1900-01-01,-,1.2,-,-,La Chaux-de-Fonds,1900,1
1,CDF,1900-01-02,-,0.0,-,-,La Chaux-de-Fonds,1900,1
2,CDF,1900-01-03,-,27.4,-,-,La Chaux-de-Fonds,1900,1
3,CDF,1900-01-04,-,13.7,-,-,La Chaux-de-Fonds,1900,1
4,CDF,1900-01-05,-,2.5,-,-,La Chaux-de-Fonds,1900,1
...,...,...,...,...,...,...,...,...,...
212720,STG,2023-04-05,-,0.0,721,1.1,St. Gallen,2023,4
212721,STG,2023-04-06,-,0.0,607,5.1,St. Gallen,2023,4
212722,STG,2023-04-07,-,1.1,125,5.8,St. Gallen,2023,4
212723,STG,2023-04-08,-,12.4,0,4.0,St. Gallen,2023,4


There we go, the final data. Let's check wether it was actually colder than usual this year.

> Easter is a “movable feast,” so it doesn't happen on the same date from year to year. In the Gregorian calendar, it is always observed on a Sunday between March 22 and April 25.

Surely, as always, there's a package that covers that headache.

In [210]:
import holidays

tmp = holidays.country_holidays('CH', subdiv='ZH', years=range(1960, 2023 + 1))

In [211]:
swiss_holidays = pd.DataFrame({'date': list(tmp.keys()),
                               'day': list(tmp.values())})

swiss_holidays

,date,day
0,1960-01-01,Neujahrestag
1,1960-01-02,Berchtoldstag
2,1960-04-17,Ostern
3,1960-04-15,Karfreitag
4,1960-04-18,Ostermontag
...,...,...
762,2023-05-28,Pfingsten
763,2023-05-29,Pfingstmontag
764,2023-08-01,Nationalfeiertag
765,2023-12-25,Weihnachten


From this, I can left join the easter days onto the original data:

In [212]:
easter_holidays = (swiss_holidays
                   .query('day == "Ostern"')
                   .assign(date = lambda x: pd.to_datetime(x['date']))
                   .reset_index(drop=True)
                   )

easter_holidays.head()

,date,day
0,1960-04-17,Ostern
1,1961-04-02,Ostern
2,1962-04-22,Ostern
3,1963-04-14,Ostern
4,1964-03-29,Ostern


In [216]:
data = (data
 .merge(easter_holidays, how='left')
 .dropna(subset='day')
 )

data.head()

,station/location,date,Bewoelkung,Niederschlag,Sonnenscheindauer,Temperatur,Station,year,month,day
0,CDF,1960-04-17,100,7.1,0,1.2,La Chaux-de-Fonds,1960,4,Ostern
1,CDF,1961-04-02,77,0.1,270,8.4,La Chaux-de-Fonds,1961,4,Ostern
2,CDF,1962-04-22,60,5.0,330,12,La Chaux-de-Fonds,1962,4,Ostern
3,CDF,1963-04-14,20,0.0,534,5.1,La Chaux-de-Fonds,1963,4,Ostern
4,CDF,1964-03-29,100,0.0,18,1.8,La Chaux-de-Fonds,1964,3,Ostern


All done, let's write the data to csv and then make some charts in R:

In [230]:
data.to_csv('weather_data_easter.csv', index=False)